In [1]:
!pip install plotly


Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: C:\Users\hanan\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [2]:
!pip install pyarrow


Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: C:\Users\hanan\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [3]:
!pip install --upgrade kaleido


Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: C:\Users\hanan\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [22]:
from pathlib import Path
import numpy as np
import pandas as pd
import plotly.express as px

DATA = Path("../data/processed/analytics_table.parquet")
FIGS = Path("../reports/figures")
FIGS.mkdir(parents=True, exist_ok=True)

def save_fig(fig, path: Path, *, scale: int = 2) -> None:
    """Save a Plotly figure to disk (requires `kaleido`)."""
    path.parent.mkdir(parents=True, exist_ok=True)
    fig.write_image(str(path), scale=scale)
#-------------------------------------------------------------------
df = pd.read_parquet(DATA)
#df
print("rows:", len(df), "cols:", len(df.columns))
print(df.dtypes.head(15))
print(df.isna().sum().sort_values(ascending=False).head(10))


rows: 5 cols: 21
order_id                string[python]
user_id                 string[python]
amount                         Float64
quantity                         Int64
created_at         datetime64[ns, UTC]
status                          object
status_clean                    object
amount_isna                       bool
quantity_isna                     bool
created_at_isna                   bool
status_isna                       bool
date                            object
year                           float64
month                          float64
day                            float64
dtype: object
amount                1
created_at            1
quantity              1
year                  1
date                  1
hour                  1
day_name              1
amount__is_outlier    1
amount_winsor         1
day                   1
dtype: int64


In [7]:
#: Revenue by country 
rev = (
    df.groupby("country", dropna=False)
    .agg(
        n=("order_id","size"),
        revenue=("amount","sum"),
        aov=("amount","mean"),
    )
    .reset_index()
    .sort_values("revenue", ascending=False)
)
rev.head()

#print (rev)



,country,n,revenue,aov
1,sa,4,145.5,36.375
0,ae,1,0.0,<NA>


In [8]:
pip install -U kaleido


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: C:\Users\hanan\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [18]:
%pip install -U nbformat ipython

Defaulting to user installation because normal site-packages is not writeable
  Using cached jsonschema-4.25.1-py3-none-any.whl.metadata (7.6 kB)
  Using cached attrs-25.4.0-py3-none-any.whl.metadata (10 kB)
  Using cached jsonschema_specifications-2025.9.1-py3-none-any.whl.metadata (2.9 kB)
  Using cached referencing-0.37.0-py3-none-any.whl.metadata (2.8 kB)
Using cached jsonschema-4.25.1-py3-none-any.whl (90 kB)
Using cached attrs-25.4.0-py3-none-any.whl (67 kB)
Using cached jsonschema_specifications-2025.9.1-py3-none-any.whl (18 kB)
Using cached referencing-0.37.0-py3-none-any.whl (26 kB)

   ----------- ---------------------------- 2/7 [attrs]
   ----------------- ---------------------- 3/7 [referencing]
   ---------------------------- ----------- 5/7 [jsonschema]
   ---------------------------------- ----- 6/7 [nbformat]
   ---------------------------------------- 7/7 [nbformat]

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: C:\Users\hanan\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [ ]:
import plotly.express as px

fig = px.bar( rev,  x="country", y="revenue", title="Revenue by country (all data)")
fig.update_layout(title={"x": 0.02})
fig.update_xaxes(title_text="Country")
fig.update_yaxes(title_text="Revenue (sum of amount)")
fig
save_fig(fig, FIGS / "revenue_by_country.png")


In [27]:
#Revenue trend (monthly)
if "month" not in df.columns and "created_at" in df.columns:
    dt = pd.to_datetime(df["created_at"], errors="coerce", utc=True)
    df["month"] = dt.dt.to_period("M").astype("string")
trend = (
    df.groupby("month", dropna=False)
      .agg(
        revenue=("amount", "sum"),
        n=("order_id", "size"),      
      )
      .reset_index()
      .sort_values("month")
)

trend.head()

,month,revenue,n
0,12.0,45.5,4
1,NaN,100.0,1


In [29]:
fig = px.line(trend, x="month", y="revenue", title="Revenue over time (monthly)")
fig.update_layout(title={"x":0.02})
fig.update_xaxes(title_text="Month")
fig.update_yaxes(title_text="Revenue")
fig
save_fig(fig, FIGS / "revenue_trend_monthly.png")


In [52]:
# Amount distribution (winsorized)
if "amount_winsor" not in df.columns and "amount" in df.columns:
    lo, hi = df["amount"].quantile([0.01, 0.99])
    df["amount_winsor"] = df["amount"].clip(lo, hi)
fig = px.histogram(df, x="amount_winsor", nbins=30, title="Order amount distribution (winsorized)")
fig.update_layout(title={"x": 0.02})
fig.update_xaxes(title_text="Amount (winsorized)")
fig.update_yaxes(title_text="Number of orders")
fig
save_fig(fig, FIGS / "amount_hist_winsor.png")

In [51]:
def bootstrap_diff_means(a: pd.Series, b: pd.Series, *, n_boot: int = 2000, seed: int = 0) -> dict:
    rng = np.random.default_rng(seed)
    a = pd.to_numeric(a, errors="coerce").dropna().to_numpy()
    b = pd.to_numeric(b, errors="coerce").dropna().to_numpy()
    assert len(a) > 0 and len(b) > 0, "Empty group after cleaning"
    diffs = []
    for _ in range(n_boot):
        sa = rng.choice(a, size=len(a), replace=True)
        sb = rng.choice(b, size=len(b), replace=True)
        diffs.append(sa.mean() - sb.mean())
    diffs = np.array(diffs)
    return {
        "diff_mean": float(a.mean() - b.mean()),
        "ci_low": float(np.quantile(diffs, 0.025)),
        "ci_high": float(np.quantile(diffs, 0.975)),
    }
d = df.assign(is_refund=df["status_clean"].eq("refund").astype(int))
a = d.loc[d["country"].eq("sa"), "is_refund"]
b = d.loc[d["country"].eq("ae"), "is_refund"]
print("n-sa:", len(a), "n-ae:", len(b))

res = bootstrap_diff_means(a, b, n_boot=2000, seed=0)
print(res)


n-sa: 4 n-ae: 1
{'diff_mean': -1.0, 'ci_low': -1.0, 'ci_high': -1.0}
